### 1. 問題分析結果

#### 競技プログラミング視点

* **最速手法**: 値→添字のハッシュ（`dict[int, int]`）を用いた**1パス**。各要素 `x` に対し補数 `target-x` が既出かを **平均 O(1)** で照会し、見つかったら即返す → **時間 O(n)**。
* **メモリ最小化**: 最初の出現のみ格納（同値が複数あっても最左のみ保持）。配列は非破壊・イミュータブルな読み取り。

#### 業務開発視点

* **型安全性/可読性**: `List[int]` 明示・戻り値も `List[int]` 固定で *pylance* 互換。責務が明確なローカル変数名（`seen`, `need`）。
* **エラーハンドリング**: LeetCode 前提では省略可だが、実務版では型・長さ検証を行い `TypeError/ValueError` を送出するメソッドも併設。

#### Python特有分析

* **CPython特性**: `dict` はハッシュテーブルのC実装で高速（平均 O(1)）。`enumerate` により Python レベルのインデックス管理コストを削減。
* **GIL**: 単一スレッドCPUバウンドのため影響無し。I/Oも絡まない。
* **内蔵最適化**: ループ内のローカル変数束縛・属性/グローバル参照の回避でバイトコード最適化が乗りやすい。

---

### 2. アルゴリズム比較表

| アプローチ                | 時間計算量      | 空間計算量 | Python実装コスト | 可読性 | 標準ライブラリ活用           | CPython最適化 | 備考                   |
| -------------------- | ---------- | ----- | ----------- | --- | ------------------- | ---------- | -------------------- |
| 方法A：ハッシュ1パス（`dict`）  | **O(n)**   | O(n)  | 低           | 高   | `dict`, `enumerate` | **適**      | 最速・最小定数因子            |
| 方法B：ソート＋二ポインタ（元添字保持） | O(n log n) | O(n)  | 中           | 中   | `sorted`            | 適          | (値, 添字) ペア生成が必要      |
| 方法C：二重ループ            | O(n²)      | O(1)  | 低           | 高   | なし                  | 不適         | 小規模のみ妥当、Follow-up不満足 |

---

### 3. 採用アルゴリズムと根拠

* **選択**: 方法A（ハッシュ1パス）
* **理由**:

  * 計算量で最良（**O(n)**）。一意解保証により見つかった時点で即 return。
  * CPython での `dict` はC実装で高速、かつコードは短く保守性も高い。
  * 型注釈が簡潔で *pylance* と相性が良い（`List[int]` → `List[int]`）。
* **Python最適化戦略**:

  * `for i, x in enumerate(nums)` のローカル束縛で高速化。
  * `dict.get(need)` の結果を一時変数に保持し二度引きを避ける。
  * 既出位置は**最初の出現のみ**保持（重複値でバグらない）。

---

### 3. 実装パターン

* **競技向け最適化版**（LeetCode本体が呼ぶ `twoSum`）: 入力は正当と仮定し最短・最速で実装。
* **業務向け堅牢版**（補助メソッド）: 型/長さ検証を行い、安全側に倒す。

---

### 4. 検証

* **境界値**: `nums` 長さ最小の 2、同値ペア（例: `[3,3], 6`）、負数や大きな値の混在、同値が複数あるケース（最左保持で正常動作）。
* **型チェック**: `List[int]` のみ許容（*pylance* で不正引数を検出可能）。

---

## 実装（LeetCode / Class 形式, CPython 3.11, 型注釈あり）

```python
from typing import List, Sequence


class Solution:
    """
    Two Sum solver.

    - twoSum: 競技プログラミング向け最適化（LeetCode が呼び出す想定）
    - two_sum_production: 業務向け（防御的検証あり）の参考実装
    """

    def twoSum(self, nums: List[int], target: int) -> List[int]:
        """
        ハッシュ1パスで解く最速実装（LeetCode前提: 正当入力）

        Args:
            nums: 整数配列
            target: 目標和

        Returns:
            条件を満たす2インデックス（任意順）

        Time Complexity:
            O(n), where n = len(nums)
        Space Complexity:
            O(n) in the worst case (ハッシュに最大 n-1 件保持)
        """
        seen: dict[int, int] = {}  # value -> first index
        for i, x in enumerate(nums):
            need = target - x
            j = seen.get(need)
            if j is not None:
                return [j, i]
            # 同じ要素を2度使わないため、先に照会→後で登録
            if x not in seen:
                seen[x] = i
        # 問題の前提では到達しないが、型整合のため固定値を返す
        return [-1, -1]

    # ---- 参考：業務向けの堅牢版（LeetCodeでは使用されない） ----
    def two_sum_production(self, nums: Sequence[int], target: int) -> List[int]:
        """
        防御的検証ありの実務版（参考）。
        Raises を積極的に使い、契約違反を明示。

        Raises:
            TypeError: 非整数が混入している場合
            ValueError: 長さが制約外の場合
        """
        # 1) 型と値の検証
        if not isinstance(nums, Sequence):
            raise TypeError("nums must be a sequence of ints")
        n = len(nums)
        if n < 2:
            raise ValueError("nums must contain at least two elements")
        # LeetCode の制約に合わせるなら 2 <= n <= 10**4 程度でよいが任意
        for idx, v in enumerate(nums):
            if not isinstance(v, int):
                raise TypeError(f"nums[{idx}] must be int, got {type(v).__name__}")
        if not isinstance(target, int):
            raise TypeError("target must be int")

        # 2) メインアルゴリズム（twoSum と同一ロジック）
        seen: dict[int, int] = {}
        for i, x in enumerate(nums):
            need = target - x
            j = seen.get(need)
            if j is not None:
                return [j, i]
            if x not in seen:
                seen[x] = i

        # 本来到達しない前提だが、契約的に例外化するなら以下
        raise ValueError("No valid two-sum solution found")

Analyze Complexity
Runtime 1 ms
Beats 57.34%
Memory 18.92 MB
Beats 33.85%

```

**要点まとめ**

* `dict` 1パスで **O(n)** / **O(n)**、CPythonのC実装で高速。
* *pylance* フレンドリーな厳密型注釈（`List[int]` → `List[int]`）。
* LeetCode 用は最短実装、実務版はバリデーションを追加して安全性を担保。

Two Sum はもう **定数因子勝負**なので伸びしろは小さめですが、LeetCode 実行環境向けの**マイクロ最適化**で数％の改善が出ることがあります。

---

## 伸ばせる可能性があるポイント

1. **“補数を辞書に入れる” 方式**

* ループで毎回 `need = target - x` を計算するのは同じですが、
  「`x` が既に辞書にあるか？」で判定でき、**減算→辞書アクセスの回数を1回**にできます。
  （従来：`get(need)` → miss なら `set(x)`、提案：`if x in seen` → miss なら `seen[target-x]=i`）

2. **メソッドのローカル束縛**

* `seen.get` / `seen.__setitem__` / `seen.__contains__` をローカルに束縛すると**属性解決のオーバーヘッド**を削減できます（CPythonの定番小技）。

3. **分岐の削減**

* `in` 判定は `get` の `None` 判定よりわずかに有利なことがあります（キー 0 の扱いも明確）。

4. **例外パスを使わない**

* `try/except KeyError` は早いケースもありますが、ミスヒット頻度が高い本問題では `in`/`get` の方が安定しがち。

> メモリについては辞書必須のため劇的改善は難しいですが、**初出のみ格納**（重複時に上書きしない）は継続が最良です。

---

## マイクロ最適化版（LeetCode / Class 形式, CPython 3.11, 型注釈あり）

```python
from typing import List


class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
        """
        Complement-first dict + method binding による軽量実装
        Time: O(n), Space: O(n)
        """
        seen: dict[int, int] = {}  # key: complement(= target - value), val: index
        contains = seen.__contains__      # ローカル束縛で属性解決を削減
        setitem = seen.__setitem__        # 代入もバインドして高速化
        # ループ: 先に「x が辞書にあるか？」を見て、無ければ complement を登録
        for i, x in enumerate(nums):
            if contains(x):
                return [seen[x], i]
            setitem(target - x, i)
        return [-1, -1]  # 問題前提では到達しない

Analyze Complexity
Runtime 2 ms
Beats 52.41%
Memory 19.18 MB
Beats 10.37%

```

### 変更点の効果

* **辞書アクセス1回化**：`x in seen` → 1回、ヒット時は即 return。
* **属性解決の削減**：`contains` / `setitem` をローカルで保持。
* **同値処理の簡潔化**：初出のみ保持は暗黙に満たされます（後続は上書きせず、常に「補数」を登録するため）。

---

## それでも伸びない場合

* LeetCode の計測は**ノイズが大きい**ため、数回提出して中央値で判断してください。
* もしメモリが気になる場合は、**コメントや余分な一時変数を減らす**程度（影響はごく小）です。

**まとめ**: 既に上位帯ですが、上記の**補数辞書 + メソッド束縛**は Python では定番の最終調整です。

